In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go

### sub-system

In [2]:
alpha = 1
delta = .25
gamma = .3
beta = -1
omega = 1
def dufffingOscillator(state,t):
    x,y = state[0],state[1]
    xdot = y
    ydot = -alpha*(x**3) - beta* x - delta*y + gamma*cos(omega*t)
    return asarray((xdot,ydot))

### coupling

In [3]:
a0,a1 = 1.5,1.2
w = 1.3

In [4]:
def coupling(stateA,stateB,t):
    x1,y1 = stateA
    x2,y2 = stateB
    xdot = a1*w*x1*cos(w*t)+ y1*(a0+a1*sin(w*t)) -y2 -(x2-x1*(a0+a1*sin(w*t)))
    ydot = a1*w*y1*cos(w*t) + alpha*x2**3 + beta*x2 + delta*y2 - gamma*cos(w*t)+(a0+a1*sin(w*t))*(gamma*cos(w*t)-alpha*x1**3-beta*x1-delta*y1) -(y2-y1*(a0+a1*sin(w*t)))
    return asarray((xdot,ydot))

### simulation

In [5]:
time = 50
delTime = .1
timeline = arange(0.0, time, delTime)

In [6]:
stateA0 = asarray([0,0])
stateB0 = asarray([0,0])
stateA,stateB = simulation.evolveAB(dufffingOscillator,dufffingOscillator,stateA0,stateB0,couplingB=coupling,time=time,delTime=delTime)

### synchronization - order parameter

In [7]:
py.iplot([go.Scatter(x=timeline,y=stateB[:,0]/stateA[:,0]),
         go.Scatter(x=timeline,y=stateB[:,1]/stateA[:,1])])

<ipython-input-7-5bf4e404101d>:1: RuntimeWarning:

invalid value encountered in true_divide

<ipython-input-7-5bf4e404101d>:2: RuntimeWarning:

invalid value encountered in true_divide



### eigenvalue evolution

In [12]:
def eigenvalue(stateA,stateB,t):
    return -(a0**2)-2*a0*a1*sin(t*w)-(a1**2)*(sin(t*w)**2)-1

In [13]:
py.iplot([go.Scatter(x=timeline,y=eigenvalue(stateA,stateB,timeline))])